## Vamos a hacer un pequeño tutorial sobre como usar AIDE

Vamos a ver una forma muy fácil y visual de como usar AIDE con los LLM de OpenAI, para ello necesitamos tener crédito en nuestra cartera de la API de OpenAI --> <a href="https://platform.openai.com/settings/organization/billing/overview">Para ver como meter saldo visitar esta página</a>

In [1]:
# IMPORTS 
from openai import OpenAI
import aide

Todas las librerías que uso en este fichero las podemos encontrar en el fichero <b>aideml-requirements.txt</b>

Hacemos uso del import de OpenAI para poder ver la cantidad de modelos a los que podemos acceder

In [31]:
lista_modelos = OpenAI().models.list().dict() # Con esto sacamos la lista de modelos y la convertimos en un diccionario

In [36]:
for modelo in lista_modelos["data"]:
    print(modelo["id"])

dall-e-3
gpt-4-1106-preview
whisper-1
gpt-4o-2024-05-13
babbage-002
dall-e-2
gpt-3.5-turbo-16k
tts-1-hd-1106
tts-1-hd
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
text-embedding-3-small
gpt-4-turbo-2024-04-09
tts-1
gpt-4-turbo
text-embedding-3-large
gpt-3.5-turbo-1106
gpt-4-0125-preview
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-0301
gpt-4-turbo-preview
tts-1-1106
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-0613
gpt-4
text-embedding-ada-002
gpt-4-1106-vision-preview
davinci-002
gpt-4-0613
gpt-4-vision-preview
gpt-4o


Esta es la salida que deberías obtener al tener un mínimo de saldo en la cuenta, hay que pensar que cada modelo tiene su propio precio por token y que podemos consultar en la propia página de la <a href="https://openai.com/api/pricing/">API de OpenAI</a>. Hay que tener en cuenta que los precios de entrada y salida son para cada millos de tokens procesados.

Una vez elegido el modelo que queremos usar tenemos dos opciones de seleccionarlo,
- (1) Cambiarlo por consola con el comando: <b>agent.code.model="modelo-a-usar"</b>

- (2) Cambiarlo en el fichero <b>config.yaml</b> de la extensión aide.

Yo uso siempre la segunda forma, para ello nos dirijimos a la carpeta de nuetro entorno virtual --> navegamos hasta la carpeta <b>lib</b> --> seleccionamos la versión, en mi caso la <b>3.10.0</b> --> buscamos la carpeta <b>aide</b> --> dentro de la carpeta <b>utils</b> encontraremos el fichero <b>config.yaml</b>.

En este fichero lo único que tenemos que cambiar son los dos parámetros llamados "model" tanto en la generación como en la validación, por defecto están asignados en "gpt-4-turbo" que es un buen modelo que no consume tanto crédito como "gpt-4o" o "gpt-4".

Ahora vamos a seleccionar algún problema medianamente complejo de Kaggle y pasárselo al AIDE.

Como primer dataset voy a usar un csv con más de 250K de pacientes de diabetes para hacer un clasificador,  no es una tarea muy complicada pero si sacar una buena accuracy y como ejemplo está más que bien

In [37]:
# Definimos las variables que vamos a utilizar
data_dir = "./archive/"
goal = """
This dataset contains 3 files:

    diabetes _ 012 _ health _ indicators _ BRFSS2015.csv is a clean dataset of 253,680 survey responses to the CDC's BRFSS2015. The target variable Diabetes_012 has 3 classes. 0 is for no diabetes or only during pregnancy, 1 is for prediabetes, and 2 is for diabetes. There is class imbalance in this dataset. This dataset has 21 feature variables
    diabetes _ binary _ 5050split _ health _ indicators _ BRFSS2015.csv is a clean dataset of 70,692 survey responses to the CDC's BRFSS2015. It has an equal 50-50 split of respondents with no diabetes and with either prediabetes or diabetes. The target variable Diabetes_binary has 2 classes. 0 is for no diabetes, and 1 is for prediabetes or diabetes. This dataset has 21 feature variables and is balanced.
    diabetes _ binary _ health _ indicators _ BRFSS2015.csv is a clean dataset of 253,680 survey responses to the CDC's BRFSS2015. The target variable Diabetes_binary has 2 classes. 0 is for no diabetes, and 1 is for prediabetes or diabetes. This dataset has 21 feature variables and is not balanced
    
    
Can survey questions from the BRFSS provide accurate predictions of whether an individual has diabetes?
What risk factors are most predictive of diabetes risk?
Can we use a subset of the risk factors to accurately predict whether an individual has diabetes?
Can we create a short form of questions from the BRFSS using feature selection to accurately predict if someone might have diabetes or is at high risk of diabetes?

"""
eval="Use RMSLE, f1-score and accuracy"

Con estas declaraciones podemos ya probar el AIDE, para declaralo simplemente llamamos al constructor y le pasamos los 3 parámetros que acabamos de definir.

El tercer parámetro <b>goal</b> es opcional pero siempre está bien poner como queremos que el modelo actue.

In [38]:
experimento = aide.Experiment(
    data_dir=data_dir,
    goal=goal,
    eval=eval
)

Una vez creada la instancia solo tenemos que hacerla correr, hay que tener en cuenta que el número de pasos que pongamos incrementará las iteraciones que el modelo intenta mejorar su solución propuesta y por tanto aumentará la cuota de la consulta.*

In [39]:
mejor_solucion = experimento.run(steps=10)

Plan + code extraction failed, retrying...
Plan + code extraction failed, retrying...


/Users/alejandro/Documents/Repositorios/alejandromadrid/.aide-venv/lib/python3.10/site-packages/aide/utils/tree_export.py:34: RuntimeWarning: invalid value encountered in divide
  layout = (layout - layout.min(axis=0)) / (layout.max(axis=0) - layout.min(axis=0))


AssertionError: 

Una vez ejecutado vamos a sacar por pantalla cuál es la mejor <b>matrix validation</b> que nos indica cuan buena es la solución propuesta así como el código generado --> este se guarda en un fichero dentro del directorio por si no queremos mostrar por pantalla el código.

Además del código, AIDE genera un documento .html con un árbol para que podamos ver como ha desarrollado la o las soluciones propuestas así como por que ha elegido esa (interpretable)

In [ ]:
print(f"La mejor solución tiene una validación de --> {mejor_solucion.valid_metric}")
print(f"El código de la mejor propuesta es: \n {mejor_solucion.code}")

Esta ejecución ha costado <label style="color:red">$0.78</label>

Como esta ejecución ha fallado vamos a probar con otra forma de plantear el problema ya que la anterior ha fallado por no ser capaz de comprender todo lo necesario creo yo.

In [41]:
# Definimos las variables que vamos a utilizar
data_dir = "./archive/"
goal = """
This dataset has 2 files: 
    (1) diabetes_012_health_indicators_BRFSS2015.csv : clean dataset of 253680 survey. We have 3 classes (0) no diabetes, (1) prediabetes, (2) diabetes
    (2) diabetes_binary_5050split_health_indicators_BRFSS2015.csv: clean dataset of 70692 survey. It has an equal 50-50 for non-diabetes and (pre-diabetes + diabetes). Has 2 classes (0) no diabetes, (1) pre or diabetes 
Can we create a predictor to estimate if our patient have or not diabetes, you can use the first dataset to split it into 80-20 training test to test if the classifier is good and try it on the second dataset.
"""
eval="f1-score"

Para hacerlo más sencillo y masticable por aide he eliminado el tercer fichero csv que usamos como datos adicionales, así como acortar y simplificar las explicaciones de los datasets y cambiar las 4 preguntas que teníamos anteriormente como una oración. 

También hemos cambiado la métrica de evaluación a <b>f1-score</b> para poder tener la información tanto de la accuracy como el recall ya que es una media armónica de ambas.

In [42]:
experimento2 = aide.Experiment(
    data_dir=data_dir,
    goal=goal,
    eval=eval
)

Repetimos todo otra vez a ver si es la indicada. 

Las ejecuciones de estos tipos suelen durar de 5 a 15 según las pruebas que he realizado por lo que es normal que tarde.

In [44]:
mejor_solucion = experimento2.run(steps=10)

/Users/alejandro/Documents/Repositorios/alejandromadrid/.aide-venv/lib/python3.10/site-packages/aide/utils/tree_export.py:34: RuntimeWarning: invalid value encountered in divide
  layout = (layout - layout.min(axis=0)) / (layout.max(axis=0) - layout.min(axis=0))


AssertionError: 

In [45]:
print(f"La mejor solución tiene una validación de --> {mejor_solucion.valid_metric}")
print(f"El código de la mejor propuesta es: \n {mejor_solucion.code}")

NameError: name 'mejor_solucion' is not defined

Parece ser que no es capaz de guardar el fichero donde toca pero si que tengo el documento .py 

Este es el contenido de ejecutar aide --> guardado en logs --> 2 --> best_solution.py

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_log_error, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the datasets
df1 = pd.read_csv("./archive/diabetes_012_health_indicators_BRFSS2015.csv")
df2 = pd.read_csv("./archive/diabetes_binary_5050split_health_indicators_BRFSS2015.csv")

# Prepare the first dataset
X = df1.drop("Diabetes_012", axis=1)
y = df1["Diabetes_012"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the RandomForest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Calculate the evaluation metrics
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average="macro")
accuracy = accuracy_score(y_test, y_pred)

print(f"RMSLE: {rmsle}, F1-Score: {f1}, Accuracy: {accuracy}")

# Apply the model to the second dataset
X_new = df2.drop("Diabetes_binary", axis=1)
X_new_scaled = scaler.transform(X_new)
y_new_pred = model.predict(X_new_scaled)

# Save the predictions
submission = pd.DataFrame({"Diabetes_binary": y_new_pred})
submission.to_csv("./archive/submission.csv", index=False)


RMSLE: 0.42078933524206985, F1-Score: 0.3965432034760695, Accuracy: 0.8410990223904131


#### El total de esta ejecución ha sido de <label style="color:red">$0.63</label>